In [2]:
!pip install -U numba
!pip install fastdist

  Obtaining dependency information for numba from https://files.pythonhosted.org/packages/b5/de/e2ef933a99c502d2ec5dda1a43a74ab98b1b606c0ff17422d42c62a6f00f/numba-0.58.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for llvmlite<0.42,>=0.41.0dev0 from https://files.pythonhosted.org/packages/fc/7a/c6741000d767fed4b339fcd4fa65afbc5fe776473d5f9e9c41eceab0a7c6/llvmlite-0.41.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.2 MB/s eta 0:00:0031m8.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 12.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.38.0
    Uninstalling llvmlite-0.38.0:
      Successfully uninstalled llvmlite-0.38.0
  Attempting uninstall: numba
    Found existing installation: numba 0.55.1
    Uninstalling numba-0.55.1:
      Successfully uninstalled numba-0.55.1

[notice] A new release of pip is avail

In [1]:
import pandas as pd

train_df = pd.read_csv('data/autsl/train_labels.csv', header=None, names=['path', 'label'])
val_df = pd.read_csv('data/autsl/val_labels.csv', header=None, names=['path', 'label'])
test_df = pd.read_csv('data/autsl/test_labels.csv', header=None, names=['path', 'label'])

In [5]:
pd.read_csv('data/autsl/classes.csv')

,ClassID,ClassName_tr,EN
0,0,abla,sister
1,1,acele,hurry
2,2,acikmak,hungry
3,3,afiyet_olsun,enjoy_your_meal
4,4,agabey,brother
...,...,...,...
221,221,yol,road
222,222,yorgun,tired
223,223,yumurta,egg
224,224,zaman,time


## Classification

In [1]:
from lib.config import *
from lib.dataset.dataloader import PoseTopologyDataset, BosphorusSign22kDataset, DataLoader, collate_fn_pose_enc, AUTSLDataset

print('Loading dataset...')
BSIGN = BosphorusSign22kDataset(DATASET_FILE_PATH='data/bsign22k/BosphorusSign22k.csv')
print('Loaded dataset...')

training_labels = BSIGN.train_labels.tolist()
training_data = PoseTopologyDataset(BSIGN.train_videos, training_labels, 'datasets/train')
train_dataloader = DataLoader(
    training_data,
    collate_fn=collate_fn_pose_enc, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

testing_labels = BSIGN.test_labels.tolist()
testing_data = PoseTopologyDataset(BSIGN.test_videos, testing_labels, 'datasets/test')
test_dataloader = DataLoader(
    testing_data,
    collate_fn=collate_fn_pose_enc, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

next(iter(train_dataloader))


AUTSL = AUTSLDataset(POSE_DATA_PATH='data/autsl/mmpose-full')
print('Loaded dataset...')

training_labels = AUTSL.train_labels.tolist()
training_data = PoseTopologyDataset(BSIGN.train_videos, training_labels, 'datasets/train')
train_dataloader = DataLoader(
    training_data,
    collate_fn=collate_fn_pose_enc, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

testing_labels = AUTSL.test_labels.tolist()
testing_data = PoseTopologyDataset(BSIGN.test_videos, testing_labels, 'datasets/test')
test_dataloader = DataLoader(
    testing_data,
    collate_fn=collate_fn_pose_enc, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

next(iter(train_dataloader))


Loading dataset...
Loaded dataset...


FileNotFoundError: [Errno 2] No such file or directory: 'data/autsl/mmpose-full/0361/User_3_001.pickle'

(tensor([[[[[0.0000e+00, 1.1472e+01, 2.1996e+01,  ..., 2.2604e+01,
             2.2390e+01, 2.7157e+01],
            [1.1472e+01, 0.0000e+00, 1.0524e+01,  ..., 1.1132e+01,
             1.0918e+01, 1.5685e+01],
            [2.1996e+01, 1.0524e+01, 0.0000e+00,  ..., 6.0788e-01,
             3.9380e-01, 5.1606e+00],
            ...,
            [2.2604e+01, 1.1132e+01, 6.0788e-01,  ..., 0.0000e+00,
             2.1408e-01, 4.5527e+00],
            [2.2390e+01, 1.0918e+01, 3.9380e-01,  ..., 2.1408e-01,
             0.0000e+00, 4.7668e+00],
            [2.7157e+01, 1.5685e+01, 5.1606e+00,  ..., 4.5527e+00,
             4.7668e+00, 0.0000e+00]],
 
           [[0.0000e+00, 7.1906e-01, 2.8001e+00,  ..., 4.4288e+01,
             4.4127e+01, 1.7132e+01],
            [7.1906e-01, 0.0000e+00, 2.0811e+00,  ..., 4.3569e+01,
             4.3408e+01, 1.6413e+01],
            [2.8001e+00, 2.0811e+00, 0.0000e+00,  ..., 4.1488e+01,
             4.1327e+01, 1.4332e+01],
            ...,
            [4.428

In [2]:
from lib.model.posemae import Sign2Gloss

model = Sign2Gloss()
gloss_model = model.model

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from lib.training.classification import ClassificationTrainer


trainer = ClassificationTrainer(gloss_model,
                                train_dataloader,
                                test_dataloader,
                                device='cpu',
                                learning_rate=LEARNING_RATE)

In [4]:
trainer.train()

0it [00:00, ?it/s]

0it [00:00, ?it/s]


AttributeError: 'numpy.int64' object has no attribute 'split'

## Translation

In [1]:
from lib.config import *
from lib.dataset.dataloader import PoseTopologyDataset, BosphorusSign22kDataset, ASLFingerSpellingDataset, collate_fn_pose_enc

ASL = ASLFingerSpellingDataset()
data = ASL.dataset

/Users/karahansahin/Desktop/Thesis Repos/SignViViT/lib/dataset/dataloader.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.path = dataset.path.apply(lambda x: x.split('/')[1])


In [2]:
from lib.model.translation import Sign2Text

model = Sign2Text()
translation = model.model

Some weights of VivitModel were not initialized from the model checkpoint at google/vivit-b-16x2 and are newly initialized: ['vivit.pooler.dense.weight', 'vivit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.6.crossattention.c_proj.weight', 'h.0.crossattention.q_attn.weight', 'h.8.crossattention.q_attn.bias', 'h.4.crossattention.c_proj.bias', 'h.10.crossattention.c_attn.bias', 'h.0.ln_cross_attn.bias', 'h.3.crossattention.c_proj.weight', 'h.7.crossattention.q_attn.bias', 'h.1.crossattention.c_proj.bias', 'h.10.crossattention.c_proj.weight', 'h.2.crossattention.c_attn.weight', 'h.7.ln_cross_attn.weight', 'h.11.crossattention.c_attn.weight', 'h.9.crossattention.q_attn.bias', 'h.11.crossattention.q_attn.bias', 'h.6.ln_cross_attn.bias', 'h.2.crossattention.q_attn.bias', 'h.10.crossattention.

In [ ]:
from lib.training.translation import Seq2SeqTrainer

trainer = Seq2SeqTrainer(translation,
                         train_dataset=data['train'],
                         val_dataset=data['val'])

In [4]:
trainer.train()

NameError: name 'trainer' is not defined